In [20]:
import findspark
from pyspark.sql import SparkSession
import math

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [21]:
sc = spark.sparkContext

Cree un RDD llamado lenguajes que contenga los siguientes lenguajes de programación: Python, R, C, Scala, Ruby y SQL.

    1.-Obtenga un nuevo RDD a partir del RDD lenguajes donde todos los lenguajes de programación estén en mayúsculas.
    2.-Obtenga un nuevo RDD a partir del RDD lenguajes donde todos los lenguajes de programación estén en minúsculas.
    3.-Cree un nuevo RDD que solo contenga aquellos lenguajes de programación que comiencen con la letra R.

In [22]:
rdd_lenguajes = sc.parallelize(['Python', 'R', 'C', 'Scala', 'Ruby', 'SQL'])
rdd_lenguajes.collect()

['Python', 'R', 'C', 'Scala', 'Ruby', 'SQL']

In [23]:
# 1
rdd_lenguajes_may = rdd_lenguajes.map(lambda x: x.upper())
rdd_lenguajes_may.collect()

['PYTHON', 'R', 'C', 'SCALA', 'RUBY', 'SQL']

In [24]:
#  2
rdd_lenguajes_min = rdd_lenguajes.map(lambda x: x.lower())
rdd_lenguajes_min.collect()

['python', 'r', 'c', 'scala', 'ruby', 'sql']

In [25]:
# 3
rdd_lenguajes_r = rdd_lenguajes.filter(lambda x: x.startswith('R'))
rdd_lenguajes_r.collect()

['R', 'Ruby']

Cree un RDD llamado pares que contenga los números pares existentes en el intervalo 20-30.

    1.-Cree el RDD llamado sqrt, este debe contener la raíz cuadrada de los elementos que componen el RDD pares.
    2.-Obtenga una lista compuesta por los números pares en el intervalo 20-30 y sus respectivas raíces cuadradas. Un ejemplo del resultado deseado para el intervalo 50-60 sería la lista [50, 7.0710678118654755, 52, 7.211102550927978, 54, 7.3484692283495345, 56, 7.483314773547883, 58, 7.615773105863909, 60, 7.745966692414834].
    3.-Eleve el número de particiones del RDD sqrt a 20.
    4.-Si tuviera que disminuir el número de particiones luego de haberlo establecido en 20, ¿qué función utilizaría para hacer más eficiente su código?

In [26]:
rdd_pares = sc.parallelize(range(20, 31)).filter(lambda x: x % 2 == 0)
rdd_pares.collect()

[20, 22, 24, 26, 28, 30]

In [27]:
# 1
rdd_sqrt = rdd_pares.map(lambda x: math.sqrt(x))
rdd_sqrt.collect()

[4.47213595499958,
 4.69041575982343,
 4.898979485566356,
 5.0990195135927845,
 5.291502622129181,
 5.477225575051661]

In [28]:
# 2
rdd_sqrt_kv = rdd_pares.flatMap(lambda x: (x, math.sqrt(x)))
rdd_sqrt_kv.collect()

[20,
 4.47213595499958,
 22,
 4.69041575982343,
 24,
 4.898979485566356,
 26,
 5.0990195135927845,
 28,
 5.291502622129181,
 30,
 5.477225575051661]

In [29]:
# 3
rdd_sqrt20 = rdd_sqrt.repartition(20)
rdd_sqrt20.getNumPartitions()

20

In [30]:
# 4
# Usaría coalesce

Cree un RDD del tipo clave valor a partir de los datos adjuntos como recurso a esta lección. Tenga en cuenta que deberá procesar el RDD leído para obtener el resultado solicitado. Supongamos que el RDD resultante de tipo clave valor refleja las transacciones realizadas por número de cuentas. Obtenga el monto total por cada cuenta.

Tip: Cree su propia función para procesar el RDD leído.

In [55]:
#Esto es un rdd de texto que tendremos que transformar en números
rdd_trans = sc.textFile('./data/transacciones.txt')
rdd_trans.collect()

['(1001, 52.3)',
 '(1005, 20.8)',
 '(1001, 10.1)',
 '(1004, 52.7)',
 '(1005, 20.7)',
 '(1002, 85.3)',
 '(1004, 20.9)']

In [74]:
#Creamos una función para transformar el RDD
def text_num(rdd):
    return tuple((rdd.replace('(','').replace(')','')).split(', '))

In [76]:
rdd_trans_kv = rdd_trans.map(text_num)

# Ahora tenemos key-value, pero ambas son de tipo string
rdd_trans_kv.collect()

[('1001', '52.3'),
 ('1005', '20.8'),
 ('1001', '10.1'),
 ('1004', '52.7'),
 ('1005', '20.7'),
 ('1002', '85.3'),
 ('1004', '20.9')]

In [79]:
# Como sólo hay una key con valor 1002, no llegará a transformarlo en float porque no se le aplica la lambda de reduceByKey, 
# así que habrá que indicárselo con un map
rdd_trans_total = rdd_trans_kv.reduceByKey(lambda x,y: float(x) + float(y)).map(lambda x: (x[0], float(x[1])))
rdd_trans_total.collect()

[('1002', 85.3), ('1001', 62.4), ('1005', 41.5), ('1004', 73.6)]